In [1]:
import numpy as np
import random
from scipy.stats import norm
import openpyxl as opx
import time
from sklearn.model_selection import KFold
from itertools import product
import matplotlib.pyplot as plt

In [2]:
def gaussian_distribution(days_distribution):
    # Parámetros de la campana de Gauss deseada
    mean = days_distribution / 2
    std_dev = 8.8

    # Crear un rango de valores en el eje x
    x_values = np.linspace(1, days_distribution, days_distribution)
    y_values = norm.pdf(x_values, mean, std_dev)

    # Ajustar la campana de Gauss para que la suma de las probabilidades sea 1
    sum_y_values = np.sum(y_values)
    ajuste = (1 - sum_y_values) / days_distribution

    for i in range(len(y_values)):
        y_values[i] += ajuste

    #print("Distribución Objetivo:\n", y_values)c
    #suma = np.sum(y_values)
    #print("Suma:", suma)

    return y_values, x_values

In [3]:
# Función de inicialización de la población
def initialize_population(population_size, genome_size, remainder):
    # Crear un arreglo con valores entre 0 y 0.5
    population = [np.random.uniform(0, 0.5, genome_size) for _ in range(population_size)]

    # Ajustar algunos valores para que la suma sea igual al remanente
    for i in range(population_size):
        population[i] = population[i] / np.sum(population[i]) * remainder

    #print(np.array(population))
    #print("Suma de cada fila:", np.sum(population, axis=1))
    return np.array(population)


In [4]:
def mse_fitness(y_true, y_pred):
  # Calcula la diferencia entre los valores reales y predichos
  error = y_true - y_pred

  # Calcula el cuadrado del error
  squared_error = np.square(error)

  # Calcula la media del cuadrado del error
  mse = (1 / (1 + np.mean(squared_error)))
  return mse

In [5]:
# Cruce de un individuos 
def one_point_crossover(parent1, parent2, crossover_rate):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, len(parent1) - 1)
        #print("Punto de cruce:", crossover_point)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2
    else:
        return parent1, parent2

In [6]:
# Mutación de un individuo
def mutate(individual, mutation_rate):
    mask = np.random.rand(len(individual)) < mutation_rate
    individual[mask] = np.random.rand(sum(mask))
    return individual

In [7]:
# Algoritmo genético
def genetic_algorithm(target_distribution, population_size=50, generations=50, crossover_rate=0.8, mutation_rate=0.1, elitism_rate=0.1):
    progress_days = 10
    genome_size = len(target_distribution) - progress_days

    remainder = 1 - sum(target_distribution[:10])
    # Eliminamos los días que ya han pasado
    real_progress = target_distribution[:10]
    target_dist_to_end = target_distribution[progress_days:]

    progress_days = sum(1 for value in real_progress if value is not np.nan)
    genome_size = len(target_distribution) - progress_days
    # Eliminamos los días que ya han pasado
    target_dist_to_end = target_distribution[progress_days:]
    
    # Inicializar la población arreglos que suman el remanente para terminar las tareas
    remainder = 1 - sum(value for value in real_progress if value is not np.nan)
    population = initialize_population(population_size, genome_size, remainder)
    # Calcular la aptitud de cada individuo en la población
    fitness_values = np.array([mse_fitness(individual, target_dist_to_end) for individual in population])

    for generation in range(generations):
        elite_count = int(population_size * elitism_rate)
        elite_indices = np.argsort(fitness_values)[::-1][:elite_count]
        #print(fitness_values[np.argsort(fitness_values)])
        #print("2", fitness_values[elite_indices])
        new_generation = population[elite_indices]

        while(len(new_generation) < population_size):
            #Selección por torneo de 16 individuos
            selected_indices = random.sample(range(len(fitness_values)), 16)
            
            while len(selected_indices) > 2:
                #print(selected_indices)
                indx = []

                for i in range(len(selected_indices) // 2):
                    #print(fitness_values[selected_indices[i]], fitness_values[selected_indices[-(i + 1)]])
                    if fitness_values[selected_indices[i]] > fitness_values[selected_indices[-(i + 1)]]:
                        indx.append(selected_indices[i])
                    else:
                        indx.append(selected_indices[-(i + 1)])
                
                selected_indices = indx

            
            parents = population[selected_indices]
            #print("Padres:", parents)   

            # Cruzar y mutar para obtener nuevos individuos
            children = [mutate(child, mutation_rate) for child in one_point_crossover(*parents, crossover_rate)]
            #print("Hijos:", children)
            new_generation = np.concatenate((new_generation, children))
            #print("xxx",  new_generation, "xxx")

        population = new_generation[:population_size]
        fitness_values = np.array([mse_fitness(individual, target_dist_to_end) for individual in population])

    
    # Devolver el mejor individuo al final del algoritmo
    best_index = np.argmax(fitness_values)
    best_individual = population[best_index]

    unidos = np.concatenate((real_progress, best_individual))
    return unidos

In [8]:
# Generar proyectos de prueba (simulación de datos)
def generar_proyectos(num_proyectos, num_dias, retraso):
    proyectos = []
    for _ in range(num_proyectos):
        recursos_planeados = gaussian_distribution(num_dias)[0]
        recursos_con_retraso = recursos_planeados[:10] * (1 - retraso)
        recursos_planeados[:10] = recursos_con_retraso
        proyectos.append(recursos_planeados)
    return np.array(proyectos)

In [9]:
# Función para calcular las métricas de rendimiento
def calcular_metricas(proyectos_iniciales, proyectos_finales):
    desviaciones_estandar = []
    cumplimientos_objetivos = []
    eficiencias_recursos = []

    for inicial, final in zip(proyectos_iniciales, proyectos_finales):
        desviacion = np.std(final - inicial)
        desviaciones_estandar.append(desviacion)
        
        cumplimiento = np.all(final >= 0)  # Supongamos que el objetivo es no tener recursos negativos
        cumplimientos_objetivos.append(cumplimiento)
        
        eficiencia = np.mean(final / inicial)  # Ejemplo simplificado
        eficiencias_recursos.append(eficiencia)

    return {
        "Desviacion Estandar": np.mean(desviaciones_estandar),
        "Cumplimiento de Objetivos": np.mean(cumplimientos_objetivos),
        "Eficiencia de Recursos": np.mean(eficiencias_recursos)
    }

In [10]:
# Parámetros de la simulación
num_proyectos = 10
num_dias = 44
retraso = 0.1
k = 5  # Número de folds para la validación cruzada

# Generar proyectos de prueba
proyectos_prueba = generar_proyectos(num_proyectos, num_dias, retraso)

# Rango de valores para los parámetros
pop_size_values = [45, 48, 95, 99]
cross_prob_values = [0.7275, 0.7317, 0.8565, 0.8783, 0.9182]
mut_prob_values = [0.0072, 0.0168, 0.022, 0.0269]
num_gen_values = [50, 100, 200, 300, 500]
elitism_values = [0.1, 0.01, 0.2, 0.3, 0.4]

# Realizar grid search con validación cruzada
resultados = []

# Realizar grid search con validación cruzada
mejor_puntaje = -np.inf
mejores_parametros = None

for pop_size, num_gen, cross_prob, mut_prob, elitism in product(pop_size_values, num_gen_values, cross_prob_values, mut_prob_values, elitism_values):
    print(f"Probando con Tamaño de Población: {pop_size}, Probabilidad de Cruce: {cross_prob}, Probabilidad de Mutación: {mut_prob}, Número de Generaciones: {num_gen}, Elitismo: {elitism}")
    kf = KFold(n_splits=k)
    resultados_desviacion_estandar = []
    resultados_cumplimiento_objetivos = []
    resultados_eficiencia_recursos = []
    
    for train_index, test_index in kf.split(proyectos_prueba):
        proyectos_train, proyectos_test = proyectos_prueba[train_index], proyectos_prueba[test_index]
        
        # Aplicar el algoritmo genético en el conjunto de entrenamiento
        resultados_train = [genetic_algorithm(proyecto, pop_size, num_gen, cross_prob, mut_prob, elitism) for proyecto in proyectos_train]
        
        # Evaluar el algoritmo en el conjunto de prueba
        resultados_test = [genetic_algorithm(proyecto, pop_size, num_gen, cross_prob, mut_prob, elitism) for proyecto in proyectos_test]
        
        # Calcular métricas para el conjunto de prueba
        metricas = calcular_metricas(proyectos_test, resultados_test)
        resultados_desviacion_estandar.append(metricas["Desviacion Estandar"])
        resultados_cumplimiento_objetivos.append(metricas["Cumplimiento de Objetivos"])
        resultados_eficiencia_recursos.append(metricas["Eficiencia de Recursos"])
    
    # Promediar los resultados de las métricas
    desviacion_estandar_promedio = np.mean(resultados_desviacion_estandar)
    cumplimiento_objetivos_promedio = np.mean(resultados_cumplimiento_objetivos)
    eficiencia_recursos_promedio = np.mean(resultados_eficiencia_recursos)

    # Guardar resultados
    resultados.append({
        "Parametros": (pop_size, cross_prob, mut_prob, num_gen),
        "Desviacion Estandar": desviacion_estandar_promedio,
        "Cumplimiento de Objetivos": cumplimiento_objetivos_promedio,
        "Eficiencia de Recursos": eficiencia_recursos_promedio
    })
    
    # Evaluar el puntaje global (puedes definir un criterio de evaluación compuesto)
    puntaje_global = eficiencia_recursos_promedio  # Ejemplo simplificado
    
    if puntaje_global > mejor_puntaje:
        mejor_puntaje = puntaje_global
        mejores_parametros = (pop_size, cross_prob, mut_prob, num_gen)
    
# Extraer los mejores parámetros
mejores_parametros = max(resultados, key=lambda x: x["Eficiencia de Recursos"])["Parametros"]

print("Mejores Parámetros Encontrados:")
print(f"Tamaño de la Población: {mejores_parametros[0]}")
print(f"Probabilidad de Cruce: {mejores_parametros[1]}")
print(f"Probabilidad de Mutación: {mejores_parametros[2]}")
print(f"Número de Generaciones: {mejores_parametros[3]}")
print(f"Probabilidad de Elitismo: {mejores_parametros[4]}")

# Visualización de los resultados
desviaciones = [r["Desviacion Estandar"] for r in resultados]
cumplimientos = [r["Cumplimiento de Objetivos"] for r in resultados]
eficiencias = [r["Eficiencia de Recursos"] for r in resultados]
parametros = [r["Parametros"] for r in resultados]

# Promedios de las métricas
print("Desviación Estándar Promedio:", np.mean(desviaciones))
print("Cumplimiento de Objetivos Promedio:", np.mean(cumplimientos))
print("Eficiencia de Recursos Promedio:", np.mean(eficiencias))

# Crear gráfico de barras para las métricas
fig, ax = plt.subplots(3, 1, figsize=(12, 18))

ax[0].bar(range(len(desviaciones)), desviaciones)
ax[0].set_title('Desviación Estándar Promedio')
ax[0].set_xlabel('Configuración de Parámetros')
ax[0].set_ylabel('Desviación Estándar')
ax[0].set_xticks(range(len(parametros)))
ax[0].set_xticklabels(parametros, rotation=90)

ax[1].bar(range(len(cumplimientos)), cumplimientos)
ax[1].set_title('Cumplimiento de Objetivos Promedio')
ax[1].set_xlabel('Configuración de Parámetros')
ax[1].set_ylabel('Cumplimiento de Objetivos')
ax[1].set_xticks(range(len(parametros)))
ax[1].set_xticklabels(parametros, rotation=90)

ax[2].bar(range(len(eficiencias)), eficiencias)
ax[2].set_title('Eficiencia de Recursos Promedio')
ax[2].set_xlabel('Configuración de Parámetros')
ax[2].set_ylabel('Eficiencia de Recursos')
ax[2].set_xticks(range(len(parametros)))
ax[2].set_xticklabels(parametros, rotation=90)

plt.tight_layout()
plt.show()

Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0072, Número de Generaciones: 50, Elitismo: 0.1
Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0072, Número de Generaciones: 50, Elitismo: 0.01
Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0072, Número de Generaciones: 50, Elitismo: 0.2
Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0072, Número de Generaciones: 50, Elitismo: 0.3
Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0072, Número de Generaciones: 50, Elitismo: 0.4
Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0168, Número de Generaciones: 50, Elitismo: 0.1
Probando con Tamaño de Población: 45, Probabilidad de Cruce: 0.7275, Probabilidad de Mutación: 0.0168, Número de Generaciones: 50

KeyboardInterrupt: 